In [1]:
from mpi4py import MPI
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [3]:
# Generate synthetic data for demonstration
def generate_data(num_samples, input_shape):
    return np.random.rand(num_samples, *input_shape)

In [4]:
# Define a simple convolutional neural network model
def build_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes))
    return model

In [5]:
# Train the model using parallel processing
def train_model(train_data, train_labels, epochs=10, batch_size=32):
    input_shape = train_data.shape[1:]
    num_classes = train_labels.shape[1]

    model = build_model(input_shape, num_classes)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    # Split data across processes
    data_per_process = len(train_data) // size
    start_index = rank * data_per_process
    end_index = start_index + data_per_process
    train_data_process = train_data[start_index:end_index]
    train_labels_process = train_labels[start_index:end_index]
  # Train the model
    model.fit(train_data_process, train_labels_process, epochs=epochs, batch_size=batch_size)

    return model
    

In [6]:
 if __name__ == '__main__':
    # Generate synthetic data
    num_samples = 1000
    input_shape = (28, 28, 1)  # Example image shape
    num_classes = 10  # Example number of classes
    train_data = generate_data(num_samples, input_shape)
    train_labels = np.random.randint(0, num_classes, size=(num_samples, num_classes))

    # Train the model
    trained_model = train_model(train_data, train_labels)

    # Evaluate the model (each process can evaluate its own portion of data)
    test_data = generate_data(100, input_shape)
    test_labels = np.random.randint(0, num_classes, size=(100, num_classes))
    loss, accuracy = trained_model.evaluate(test_data, test_labels)
    print(f'Process {rank}: Test loss: {loss}, Test accuracy: {accuracy}')


Epoch 1/10
32/32 [==============================] - 9s 125ms/step - loss: 2340.4375 - accuracy: 0.0730
Epoch 2/10
32/32 [==============================] - 3s 96ms/step - loss: 141901.4531 - accuracy: 0.0770
Epoch 3/10
32/32 [==============================] - 3s 94ms/step - loss: 1824427.0000 - accuracy: 0.0770
Epoch 4/10
32/32 [==============================] - 3s 106ms/step - loss: 11040781.0000 - accuracy: 0.0770
Epoch 5/10
32/32 [==============================] - 3s 98ms/step - loss: 42331112.0000 - accuracy: 0.0770
Epoch 6/10
32/32 [==============================] - 3s 102ms/step - loss: 121947968.0000 - accuracy: 0.0770
Epoch 7/10
32/32 [==============================] - 4s 110ms/step - loss: 288911936.0000 - accuracy: 0.0770
Epoch 8/10
32/32 [==============================] - 4s 114ms/step - loss: 592595968.0000 - accuracy: 0.0770
Epoch 9/10
32/32 [==============================] - 4s 115ms/step - loss: 1096832000.0000 - accuracy: 0.0770
Epoch 10/10
4/4 [=========================